<h1> Segmenting and Clustering Neighborhoods in Toronto </h1>

Assumption these libs are installed <br>
Un comment these installation if needed

In [114]:
#!pip install bs4
#!pip install HTMLParser
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [143]:
wikilink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikicontent = requests.get(wikilink).text
wikiBS = BeautifulSoup(wikicontent,'html')

# Create data frame with 3 columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']
# instantiate the dataframe
postaldf = pd.DataFrame(columns=column_names)
postaldf.head()

,PostalCode,Borough,Neighborhood


In [144]:
notasg = 'Not assigned'
# Read table by class 
wikitable = wikiBS.find('table',{'class':'wikitable'})
for row in wikitable.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    cell1 = ""
    cell2 = ""
    cell3 = ""
    
    if (len(columns) == 3):
        #postal code
        cell1 = columns[0].get_text().replace('\n', '')
        #borough
        cell2 = columns[1].get_text().replace('\n', '')
        #Continue for if borough is 'Not assigned'
        if (cell2 == notasg):
            continue
        
        #neighborhood            
        cell3 = columns[2].get_text().replace('\n', '').replace('/',',')       
        
        # finding pre and adding comma
        # if tag Pre found: pre content will added comma to front of it
        if (row.find("pre") != None) :
            pre = row.find('pre').get_text().replace('\n', '').replace('/',',')
            cell3 = cell3.replace(pre, ', ' + pre)
        
        #if neighborhood is 'Not assigned', neighborhood is same borough
        if (cell3 == notasg):
            cell3 = cell2

        postaldf = postaldf.append({'PostalCode': cell1 , 'Borough':  cell2 , 'Neighborhood':  cell3 }, ignore_index=True)
        
postaldf.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [149]:
postaldf.shape

(103, 3)

In [156]:
df=pd.read_csv('Geospatial_Coordinates.csv', delimiter = ',')
df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [158]:
postaldf['Latitude'] = ""
postaldf['Longitude'] = ""
postaldf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",,
3,M6A,North York,"Lawrence Manor , Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",,
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",,
99,M4Y,Downtown Toronto,Church and Wellesley,,
100,M7Y,East Toronto,Business reply mail Processing CentrE,,
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",,


In [170]:
for index, row in postaldf.iterrows():
    lati = df.loc[df['Postal Code'] == row['PostalCode'], 'Latitude'].values[0]
    longti = df.loc[df['Postal Code'] == row['PostalCode'], 'Longitude'].values[0]
    
    postaldf.at[index, 'Latitude'] = lati
    postaldf.at[index, 'Longitude'] = longti
postaldf.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Malvern , Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills,43.7459,-79.3522
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
